In [1]:
import os
import sys
path_base = os.getcwd()+"/.."
if path_base not in sys.path:
    sys.path.append(path_base)
    print(path_base)

import numpy as np
import mfjet 

/home/starlight/Workspace/mfjet/examples/..


In [66]:
arr_etaphi = np.array([
    [0.2, 0.3],
    [0.0, 0.3],
    [0.0, 0.3],
    [-0.5, 0.7],
])

In [67]:
arr_etaphi / 0.1

array([[ 2.,  3.],
       [ 0.,  3.],
       [ 0.,  3.],
       [-5.,  7.]])

In [68]:
np.floor(arr_etaphi / 0.1)

array([[ 2.,  2.],
       [ 0.,  2.],
       [ 0.,  2.],
       [-5.,  6.]])

In [69]:
arr_coord_idx = np.floor((arr_etaphi + (0.1*1e-6)) / 0.1).astype(int)
print(arr_coord_idx)
arr_coord_idx_min = arr_coord_idx.min(axis=0)
arr_coord_idx_max = arr_coord_idx.max(axis=0)
arr_coord_idx_rel = arr_coord_idx - arr_coord_idx_min
print(arr_coord_idx_rel)

[[ 2  3]
 [ 0  3]
 [ 0  3]
 [-5  7]]
[[7 0]
 [5 0]
 [5 0]
 [0 4]]


In [70]:
import scipy.sparse
import scipy.signal

In [71]:
np.ones(arr_coord_idx.shape[0])

array([1., 1., 1., 1.])

In [85]:
sarr_coord = scipy.sparse.coo_array(
    (
        np.ones(arr_coord_idx_rel.shape[0], dtype=int),
        arr_coord_idx_rel.T
    )
).astype(bool).astype(int)

In [86]:
print(sarr_coord)
print(sarr_coord.astype(bool))

  (0, 4)	1
  (5, 0)	1
  (7, 0)	1
  (0, 4)	True
  (5, 0)	True
  (7, 0)	True


In [73]:
filt = np.ones((3,3), dtype=int)

In [74]:
filt_binary = np.array([[1,2],[4,8]], dtype=int)

In [75]:
arr_lookup = np.array([
    [0,0,0],
    [1,4,1],
    [1,4,1],
    [2,4,0],
    [1,4,1],
    [2,4,0],
    [2,8,-2],
    [3,4,-1],
    [1,4,1],
    [2,8,-2],
    [2,4,0],
    [3,4,-1],
    [2,4,0],
    [3,4,-1],
    [3,4,-1],
    [4,0,0]
], dtype=int)[:,::-1]
print(arr_lookup.shape)

(16, 3)


In [76]:
arr_ms = scipy.signal.convolve(sarr_coord.toarray(), filt).astype(bool).astype(int)
print(arr_ms)
print(arr_ms.shape)
arr_binary = scipy.signal.convolve(arr_ms, filt_binary)
print(arr_binary.shape)

[[0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0]
 [1 1 1 0 0 0 0]
 [1 1 1 0 0 0 0]
 [1 1 1 0 0 0 0]
 [1 1 1 0 0 0 0]]
(10, 7)
(11, 8)


In [77]:
arr_mfs_local = arr_lookup[arr_binary]
print(arr_mfs_local.shape)
arr_mfs = arr_mfs_local.sum(axis=0).sum(axis=0) // 4
print(arr_mfs)

(11, 8, 3)
[ 2 28 24]


In [78]:
darr_coord = sarr_coord.toarray()

In [37]:
import time
time_start = time.time()
for _ in range(1000000):
    arr_ms = scipy.signal.convolve(sarr_coord.toarray(), filt)
    arr_ms[arr_ms>0] = 1
print("time elapsed: {:02f} s".format(time.time() - time_start))

time elapsed: 17.966556 s


In [38]:
import time
time_start = time.time()
for _ in range(1000000):
    arr_ms = scipy.signal.convolve(sarr_coord.toarray(), filt).astype(bool).astype(int)
print("time elapsed: {:02f} s".format(time.time() - time_start))

time elapsed: 17.846251 s


In [40]:
import time
time_start = time.time()
for _ in range(1000000):
    arr_ms = scipy.signal.convolve(darr_coord, filt)
    arr_ms[arr_ms>0] = 1
print("time elapsed: {:02f} s".format(time.time() - time_start))

time elapsed: 16.013347 s


In [41]:
import time
time_start = time.time()
for _ in range(1000000):
    arr_ms = scipy.signal.convolve(darr_coord, filt).astype(bool).astype(int)
print("time elapsed: {:02f} s".format(time.time() - time_start))

time elapsed: 15.578599 s


In [42]:
import time
time_start = time.time()
for _ in range(1000000):
    arr_ms = (scipy.signal.convolve(darr_coord, filt) != 0).astype(int)
print("time elapsed: {:02f} s".format(time.time() - time_start))

time elapsed: 15.717880 s
